In [286]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import math
import multiprocess as mp
import threading


In [288]:
if len(pages_df) < 3000:
    max_page = 0
    pages_df = pd.DataFrame({'break_name': [], 'url': [], 'page_number': []})
    
if len(pages_df) == 0:
    df = pd.DataFrame({'break_name': [], 'break_url': [], 'time': [], 'rating': []})

# driver.set_window_position(1000,2000)
# ChromeOptions = webdriver.ChromeOptions()
# ChromeOptions.add_argument('--disable-browser-side-navigation')
# chrome_options=ChromeOptions


try: max_page 
except: 
    driver = webdriver.Chrome()
    driver.get('https://www.surf-forecast.com/breaks/')
    breaks = driver.find_elements(By.CLASS_NAME, 'list_table')

    for i in driver.find_element(By.CLASS_NAME, 'pagination').find_elements(By.TAG_NAME, 'a'):
        try:
            cur_page = int(i.text)
            if cur_page > max_page:
                max_page = cur_page
        except:
            continue
    driver.close()
        
if max_page > pages_df['page_number'].max() or math.isnan(pages_df['page_number'].max()):
    try: start_page = int(pages_df.page_number.max()) - 1 
    except: start_page = 0; print('No Pages Yet')
        
    for page_num in range(start_page, max_page + 1):
        print('Page {}'.format(page_num) )
        driver = webdriver.Chrome()
        driver.get('https://www.surf-forecast.com/breaks?page={}'.format(page_num))
        driver.find_elements(By.CLASS_NAME, 'pagination') 
        breaks = driver.find_elements(By.CLASS_NAME, 'list_table')

        for surf_spot in breaks[0].find_elements(By.TAG_NAME, 'a'):
            pages_df = pd.concat([pages_df, pd.DataFrame({
                'break_name': [surf_spot.text]
                , 'url': [surf_spot.get_attribute('href')]
                , 'page_number': [int(page_num)]
            })], ignore_index=True)
        driver.close
        
def scrape(row):
    cur_break_df = pd.DataFrame({'break_name': [], 'break_url': [], 'time': [], 'rating': []})
    
    driver = webdriver.Chrome()
    driver.get(row['url'] + '/forecasts/latest')
    print('----')

    ratings = driver.find_elements(By.CLASS_NAME, 'star-rating__rating')
    times = driver.find_elements(By.CLASS_NAME, 'forecast-table-time__cell')

    for time, rating in zip(times, ratings):        
        cur_break_df = pd.concat([cur_break_df, pd.DataFrame({
            'break_name': [row['break_name']]
            , 'break_url': [row['url']]
            , 'time': [time.text]
            , 'rating': [rating.text]
        })], ignore_index=True)
    driver.close()

i = 0
processes = []

for index, row in pages_df.iterrows():
    i += 1
    # If this were in production Remove code until HERE and timestamp each pull
    if (df['break_name'] == row['break_name']).sum() > 40:
        continue
    print(row['break_name'])
    # HERE

    p = mp.Process(target=scrape, args=(row,))
    p.start()
    processes.append(p)
    
    if i > 10:
        break

#     df = pd.concat([df, scrape(row)], ignore_index=True)


Beer Can Beach
----
